# GMM

This notebook shows a simple example of how GMM can be used to estimate model parameters. It starts with an exactly identified case and then moves on to different ways of estimating an overidentified case (pre-defined weighting matrix, recombining the moment conditions, optimal weighting matrix).

## Load Packages and Extra Functions

The general GMM functions are from the (local) `FinEcmt_MLEGMM` module. In contrast, the functions for the moment conditions ("the model") are coded in the notebook below.

In [1]:
MyModulePath = joinpath(pwd(),"src")
!in(MyModulePath,LOAD_PATH) && push!(LOAD_PATH,MyModulePath)
using FinEcmt_OLS
using FinEcmt_MLEGMM: GMMAgbar, GMMExactlyIdentified, GMMgbarWgbar, meanV

In [2]:
#=
include(joinpath(pwd(),"src","FinEcmt_OLS.jl"))
include(joinpath(pwd(),"src","FinEcmt_MLEGMM.jl"))
using .FinEcmt_OLS
using .FinEcmt_MLEGMM: GMMAgbar, GMMExactlyIdentified, GMMgbarWgbar, meanV
=#

In [3]:
using DelimitedFiles, Statistics, NLsolve, LinearAlgebra

# Loading Data

In [4]:
x = readdlm("Data/FFmFactorsPs.csv",',',skipstart=1)   #start on line 2, column 1
x = x[:,2]         #excess market returns, in %

T = size(x,1)

388

# Exactly Identified GMM

This section describes the exactly identified GMM, that is, when we have as many moment conditions as parameters. In this case GMM is the same as the classical method of moments.

## Traditional Estimation of Mean and Variance

The next cell applies the traditional way of estimating the mean and the variance. The standard errors are text book formulas.

In [5]:
μ  = mean(x)
σ² = var(x,corrected=false)       #corrected="false" uses the 1/T formula

par_a = [μ,σ²]

printblue("Traditional estimates:\n")
xx = [par_a [sqrt((σ²/T));sqrt(2*σ²^2/T)]]
colNames = ["coef","std"]
parNames = ["μ","σ²"]
printmat(xx;colNames,rowNames=parNames)      # ; since keywords with same name

Traditional estimates:

        coef       std
μ      0.602     0.233
σ²    21.142     1.518



## GMM Point Estimates

To estimate the mean and variance of $x_{t}$, use the following moment condition

$g_{t}(\beta)=
\begin{bmatrix}
x_{t}-\mu\\
(x_{t}-\mu)^{2}-\sigma^{2}
\end{bmatrix},$

where $\beta = [\mu,\sigma^{2}]$.

The parameter values ($\mu,\sigma^2$) that make these moment conditions hold are the same as from the traditional method. It is easy to solve for these parameters when the moment conditions are linear in the parameters (as they are here). However, to facilitate adapting the code to non-linear models, we solve for the parameters by a numerical method.

In [6]:
"""
    Gmm2MomFn(par,x)

Calculate traditional 2 moment conditions for estimating [μ,σ²]. Returns a Tx2 matrix

# Input
- `par::Vector`: [μ,σ²]
- `x::Vector`:   T-vector with data

# Output
- `g::Matrix`:    Tx2, moment conditions

"""
function Gmm2MomFn(par,x)
    (μ,σ²) = (par[1],par[2])
    g      = hcat(x .- μ, abs2.(x .- μ) .- σ²)  #Tx2
    return g
end

Gmm2MomFn

### A Remark on the Code

...in the next cells.

- The `meanV()` function calculates the sample mean of each column in a matrix and returns a vector.

- The `p->meanV(Gmm2MomFn(p,x))` defines an anonymous function (in terms of the vector of parameters `p`) that returns a vector of the sample averages of the moment conditions.

- `Sol = nlsolve(p->meanV(Gmm2MomFn(p,x)),par_a)` solves for the vector `p` that makes the average moment conditions equal to `[0,0]`. (This could be done manually in this case, but perhaps not in another application.)   
    
- To extract the solution, use `Sol.zero`.

In [7]:
Sol   = nlsolve(p->meanV(Gmm2MomFn(p,x)),par_a)   #numerically solve for the estimates
par_1 = Sol.zero

printblue("GMM estimates:")
printmat(par_1;rowNames=parNames)

g    = Gmm2MomFn(par_1,x)        #Tx2, moment conditions
gbar = meanV(g)                  #2-vector with average moment conditions

printblue("Checking if mean of moment conditions = 0")
printmat(gbar;rowNames=["g₁","g₂"])

GMM estimates:
μ      0.602
σ²    21.142

Checking if mean of moment conditions = 0
g₁     0.000
g₂     0.000



## GMM Distribution

The distribution of the basic GMM estimates is

$\hat{\beta} \overset{a}{\rightarrow}N(\beta_{0},V_T),$

where 

$V_T = (D^{\prime}\Sigma^{-1}D)  ^{-1} /T,$

$\Sigma=\mathrm{Var}(\sqrt{T}\bar{g}),$

$D = \mathrm{plim}\frac{\partial\bar{g}}{\partial\beta^{\prime}}.$

(This holds for exactly identified models and models using the optimal weighting matrix.)


### A Remark on the Code

- `CovNW(g,1,1)` estimates $\Sigma$ by using the Newey-West method with one lag.

- We can notice that $D=-I_2$ for the moment conditions used above.

In [8]:
D  = -I(2)                   #Jacobian, does not really matter here
Σ  = CovNW(g,1,1)            #variance of sqrt(T)*gbar, NW with 1 lag
V_T = inv(D'inv(Σ)*D)/T

printblue("GMM estimates:\n")
printmat(par_1,sqrt.(diag(V_T));colNames,rowNames=parNames)

printstyled("Compare with the traditional estimates",color=:red,bold=true)

GMM estimates:

        coef       std
μ      0.602     0.244
σ²    21.142     2.381

Compare with the traditional estimates

## A Function for Exactly Identified GMM

The functions printed in the next cell combines the previous code into a function for getting both point estimates and standard errors. The Jacobian is calculated numerically. The result should be the same as before. (For the rest of the notebook, the code will come in functions like this.)

In [9]:
@doc2 GMMExactlyIdentified

```
GMMExactlyIdentified(GmmMomFn::Function,x,par0,m)
```

Estimates GMM coeffs and variance-covariance matrix from an exactly identified model. The Jacobian is calculated numerically.

### Input

  * `GmmMomFn::Function`:    for the moment conditions, called as `GmmMomFn(p,x)` where `p` are the coefficients and `x` is the data.
  * `x::VecOrMat`:            data
  * `par0::Vector`:           initial guess
  * `m::Int`:                 number of lags in NW covariance matrix


In [10]:
#using CodeTracking
#println(@code_string GMMExactlyIdentified(cos,[1],[1],1))

In [11]:
(par_1b,Std_1b,) = GMMExactlyIdentified(Gmm2MomFn,x,par_a,1)

printblue("Results from GMMExactlyIdentified():\n")
printmat(par_1,sqrt.(diag(V_T));colNames,rowNames=parNames)

printstyled("Compare with GMM results from above",color=:red,bold=true)

Results from GMMExactlyIdentified():

        coef       std
μ      0.602     0.244
σ²    21.142     2.381

Compare with GMM results from above

# Overidentified GMM

This section discusses an overidentified case: more moment conditions than parameters.

Warning: some of the variables (`g,Σ`, etc) are overwritten with new values.

## The Moment Conditions

If $x_{t}$ is $N(\mu,\sigma^{2})$, then the following moment conditions should
all be zero (in expectation)

$g_{t}(\beta)=
\begin{bmatrix}
x_{t}-\mu\\
(x_{t}-\mu)^{2}-\sigma^{2}\\
(x_{t}-\mu)^{3}\\
(x_{t}-\mu)^{4}-3\sigma^{4}
\end{bmatrix},$

where $\beta = [\mu,\sigma^{2}]$.

The first moment condition defines the mean $\mu$, the second defines the
variance $\sigma^{2}$, while the third and forth are the skewness and excess
kurtosis respectively.

In [12]:
"""
    Gmm4MomFn(par,x)

Calculate 4 moment conditions for estimating [μ,σ²]

# Input
- `par::Vector`: [μ,σ²]
- `x::Vector`:   T-vector with data

# Output
- `g::Matrix`:    Tx4, moment conditions

"""
function Gmm4MomFn(par,x)
  (μ,σ²) = (par[1],par[2])
  g      = hcat(x .- μ, (x .- μ).^2 .- σ², (x .- μ).^3, (x .- μ).^4 .- 3*σ²^2)    #Tx4
  return g
end

Gmm4MomFn

# Overidentified GMM: Minimizing gbar'W*gbar


The following code applies a numerical method to solve a minimization problem with the weighting matrix 

$W=
\begin{bmatrix}
1 & 0 & 0 & 0\\
0 & 1 & 0 & 0\\
0 & 0 & 0 & 0\\
0 & 0 & 0 & 0
\end{bmatrix}$

The results should be the same (or at least very close to) the previous results, since this $W$ matrix puts all weight on the first two moments. Changing $W$, for instance, by setting `W[3,3]=0.0001` will give other estimates.

We define the loss function as $\bar{g}'W\bar{g}$.

The expressions for variance-covatriance matrix are in the lecture notes.

## A Function for gbar'W*gbar Estimation

In [13]:
@doc2 GMMgbarWgbar

```
GMMgbarWgbar(GmmMomFn::Function,W,x,par0,m;SkipCovQ=false)
```

Estimates GMM coeffs and variance-covariance matrix from A*gbar. The Jacobian is calculated numerically.

### Input

  * `GmmMomFn::Function`:    for the moment conditions, called as `GmmMomFn(p,x)` where `p` are the coefficients and `x` is the data.
  * `W::Matrix`:              length(gbar)xlength(gbar)
  * `x::VecOrMat`:            data
  * `par0::Vector`:           initial guess
  * `m::Int`:                 number of lags in NW covariance matrix
  * `SkipCovQ::Bool`:         if true: the Jacobian and variance-covariance matrix are not calculated. This can be used to speed up calculations in iterative computations.


In [14]:
#println(@code_string GMMgbarWgbar(cos,[1],[1],[1],1))

In [15]:
W     = diagm(0=>[1.0,1.0,0.0,0.0])   #weighting matrix, try changing it
#W[3,3] = 0.0001
(par2,StdErr2,_,Σ2,_) = GMMgbarWgbar(Gmm4MomFn,W,x,par_a,1)

printblue("GMM estimates (gbar'W*gbar):\n")
printmat(par2,StdErr2;colNames,rowNames=parNames)

GMM estimates (gbar'W*gbar):

        coef       std
μ      0.602     0.244
σ²    21.142     2.381



# Overidentified GMM: Minimizing gbar'W*gbar, Iterating over W


The following code iterates over the weighting matrix by using $W=Σ^{-1}$, where
$\Sigma = \text{Cov}(\sqrt{T}\bar{g})$
is from the previous iteration.

### A Remark on the Code

- `(maximum(abs,par - par_old) > 1e-3) || (i < 2)` loops while the |change| in the parameters > 1e-3 (and at least once).

- `par_old = copy(par)` makes an independent copy of the `par`. This is not needed here (since `par` is overwritten further below), but often a good routine.

In [16]:
println("\niterated GMM, using optimal weighting matrix, starting with Σ from previous estimation")

(par,par_old,Σ,i) = (copy(par2),fill(Inf,length(par2)),copy(Σ2),1)

println("\n\niterating over W starting with the W choice above")
while (maximum(abs,par - par_old) > 1e-3) || (i < 2)    #require at least one iteration
    #global par, par_old, i, W, Σ    #only needed in script
    global StdErr2, D
    println("-------iteration  $i, old and new parameters--------")
    par_old = copy(par)              #update par_old
    W       = inv(Σ)
    (par,StdErr2,_,Σ,D) = GMMgbarWgbar(Gmm4MomFn,W,x,par_old,1)
    i       = i + 1
    printlnPs(par_old',"\n",par')
 end

printblue("\nGMM estimates (gbar'W*gbar, iteration over W):")
xx = [par StdErr2]
printmat(xx;colNames,rowNames=parNames,width=12)


iterated GMM, using optimal weighting matrix, starting with Σ from previous estimation


iterating over W starting with the W choice above
-------iteration  1, old and new parameters--------
     0.602    21.142          
     0.877    16.916
-------iteration  2, old and new parameters--------
     0.877    16.916          
     0.879    16.648
-------iteration  3, old and new parameters--------
     0.879    16.648          
     0.879    16.645
-------iteration  4, old and new parameters--------
     0.879    16.645          
     0.879    16.647
-------iteration  5, old and new parameters--------
     0.879    16.647          
     0.879    16.647

GMM estimates (gbar'W*gbar, iteration over W):
          coef         std
μ        0.879       0.219
σ²      16.647       1.341



In [17]:
printblue("W matrix used in the last iteration, (times 10000):\n")

momNames = ["g₁","g₂","g₃","g₄"]
printmat(W*10000,colNames=momNames,rowNames=momNames)

W matrix used in the last iteration, (times 10000):

          g₁        g₂        g₃        g₄
g₁  1525.564    39.433   -16.963    -0.674
g₂    39.433    18.778    -0.297    -0.050
g₃   -16.963    -0.297     0.306     0.012
g₄    -0.674    -0.050     0.012     0.001



In [18]:
V_T = inv(D'inv(Σ)*D)/T                      #with optimal weighting matrix

printblue("\nGMM estimates (gbar'W*gbar, iteration over W):")
xx = [par StdErr2 sqrt.(diag(V_T))]
printmat(xx;colNames=[colNames;"std ver. 2"],rowNames=parNames,width=12)

printred("Notice that the standard errors (calculated in two ways) are (almost) the same after the iterations")


GMM estimates (gbar'W*gbar, iteration over W):
          coef         std  std ver. 2
μ        0.879       0.219       0.219
σ²      16.647       1.341       1.341

Notice that the standard errors (calculated in two ways) are (almost) the same after the iterations


# Overidentified GMM: A*g = 0

The following code from estimates the parameters by combining the 4 original moment conditions in $\bar{g}$ into 2
effective moment conditions, $A\bar{g}$, where $A$ is a $2\times4$ matrix

$A=
\begin{bmatrix}
1 & 0 & 0 & 0\\
0 & 1 & 0 & 0
\end{bmatrix}$

This particular $A$ matrix implies that we use the classical
estimators of the mean and variance. Changing $A$, for instance, by setting $A[1,3]=0.001$ will give different estimates.

The next cell prints a function for doing A*gbar GMM.

In [19]:
@doc2 GMMAgbar

```
GMMAgbar(GmmMomFn::Function,A,x,par0,m)
```

Estimates GMM coeffs and variance-covariance matrix from A*gbar. The Jacobian is calculated numerically.

### Input

  * `GmmMomFn::Function`:    for the moment conditions, called as `GmmMomFn(p,x)` where `p` are the coefficients and `x` is the data.
  * `A::Matrix`:              length(p) x length(gbar)
  * `x::VecOrMat`:            data
  * `par0::Vector`:           initial guess
  * `m::Int`:                 number of lags in NW covariance matrix


In [20]:
#using CodeTracking
#println(@code_string GMMAgbar(cos,[1],[1],[1],1))

In [21]:
A = [1 0 0 0;                   #A in A*gbar=0 (here: all weight on first two moments)
     0 1 0 0]                   #try setting A[1,3] = 0.001

(par_3b,Std_3b) = GMMAgbar(Gmm4MomFn,A,x,par_a,1)


printblue("GMM estimates (A*gbar) from GMMAgbar():\n")
printmat(par_3b,Std_3b;colNames,rowNames=parNames)

GMM estimates (A*gbar) from GMMAgbar():

        coef       std
μ      0.602     0.244
σ²    21.142     2.381

